In [24]:
#!pip install preprocess
!pip install model

#!pip search model


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

def load_and_preprocess_data(filepath):
    data = pd.read_csv(used_cars.csv)
    X = data.drop(columns=['target'])
    y = data['target']
    
    # Handle categorical variables
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = LabelEncoder().fit_transform(X[col])
    
    X = StandardScaler().fit_transform(X)
    return train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

def get_model(name, params=None):
    if name == 'random_forest':
        model = RandomForestClassifier(**params)
    elif name == 'logistic_regression':
        model = LogisticRegression(**params)
    elif name == 'svm':
        model = SVC(probability=True, **params)
    else:
        raise ValueError("Unsupported model type")
    return model


In [13]:
import numpy as np
from sklearn.model_selection import cross_val_score

def random_search(model_name, X_train, y_train, param_space, n_iter=50):
    best_params = None
    best_score = -np.inf
    
    for _ in range(n_iter):
        params = {k: np.random.choice(v) for k, v in param_space.items()}
        model = get_model(model_name, params)
        score = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean()
        
        if score > best_score:
            best_score = score
            best_params = params
    
    return best_params, best_score


In [14]:
from scipy.stats import norm
import numpy as np

class BayesianOptimizer:
    def __init__(self, model_name, X_train, y_train, param_space, n_iter=50):
        self.model_name = model_name
        self.X_train = X_train
        self.y_train = y_train
        self.param_space = param_space
        self.n_iter = n_iter
        self.best_params = None
        self.best_score = -np.inf
        self.history = []

    def propose_params(self):
        if not self.history:
            return {k: np.random.choice(v) for k, v in self.param_space.items()}
        else:
            # Simplified surrogate model: using mean of the best performing configurations
            means = {k: np.mean([p[k] for p, _ in self.history]) for k in self.param_space}
            return {k: np.clip(norm.rvs(loc=means[k], scale=0.1), a_min=min(self.param_space[k]), a_max=max(self.param_space[k])) for k in self.param_space}

    def optimize(self):
        for _ in range(self.n_iter):
            params = self.propose_params()
            model = get_model(self.model_name, params)
            score = cross_val_score(model, self.X_train, self.y_train, cv=5, scoring='roc_auc').mean()
            self.history.append((params, score))
            
            if score > self.best_score:
                self.best_score = score
                self.best_params = params
                
        return self.best_params, self.best_score


In [15]:
from scipy.stats import norm
import numpy as np

class TPEOptimizer:
    def __init__(self, model_name, X_train, y_train, param_space, n_iter=50):
        self.model_name = model_name
        self.X_train = X_train
        self.y_train = y_train
        self.param_space = param_space
        self.n_iter = n_iter
        self.history = []

    def suggest(self):
        if len(self.history) < 10:
            return {k: np.random.choice(v) for k, v in self.param_space.items()}
        
        best_params = [p for p, s in self.history if s >= np.percentile([s for _, s in self.history], 80)]
        worst_params = [p for p, s in self.history if s < np.percentile([s for _, s in self.history], 80)]
        
        def acquire_best(param):
            values = [p[param] for p in best_params]
            return np.random.choice(values)
        
        def acquire_worst(param):
            values = [p[param] for p in worst_params]
            return np.random.choice(values)
        
        params = {k: acquire_best(k) if np.random.rand() > 0.5 else acquire_worst(k) for k in self.param_space}
        return params
    
    def optimize(self):
        for _ in range(self.n_iter):
            params = self.suggest()
            model = get_model(self.model_name, params)
            score = cross_val_score(model, self.X_train, self.y_train, cv=5, scoring='roc_auc').mean()
            self.history.append((params, score))
        
        best_params, best_score = max(self.history, key=lambda x: x[1])
        return best_params, best_score


In [16]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def evaluate_model(model_name, X_train, X_test, y_train, y_test, params):
    model = get_model(model_name, params)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    return fpr, tpr, roc_auc

def plot_comparison(fprs, tprs, aucs, labels):
    plt.figure(figsize=(10, 8))
    for fpr, tpr, auc_score, label in zip(fprs, tprs, aucs, labels):
        plt.plot(fpr, tpr, label=f'{label} (AUC = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()


In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_and_preprocess_data(data):
    data = load_iris()
    X = data.data
    y = data.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test


In [1]:


import numpy as np


#from sklearn.preprocessing import load_and_preprocess_data
#from model import get_model
from sklearn.optimization import random_search, BayesianOptimizer, TPEOptimizer
from sklearn.evaluation import evaluate_model, plot_comparison

# Load and preprocess data
X_train, X_test, y_train, y_test = load_and_preprocess_data('data/data.csv')

# Define parameter spaces
param_spaces = {
    'random_forest': {
        'n_estimators': range(10, 200),
        'max_depth': range(1, 32)
    },
    'logistic_regression': {
        'C': np.logspace(-4, 2, 100)
    },
    'svm': {
        'C': np.logspace(-4, 2, 100),
        'kernel': ['linear', 'rbf']
    }
}

model_name = 'random_forest'

# Random Search
baseline_params, baseline_score = random_search(model_name, X_train, y_train, param_spaces[model_name])

# Bayesian Optimization
bayesian_optimizer = BayesianOptimizer(model_name, X_train, y_train, param_spaces[model_name])
bayesian_params, bayesian_score = bayesian_optimizer.optimize()

# TPE Optimization
tpe_optimizer = TPEOptimizer(model_name, X_train, y_train, param_spaces[model_name])
tpe_params, tpe_score = tpe_optimizer.optimize()

# Evaluation
fpr_base, tpr_base, auc_base = evaluate_model(model_name, X_train, X_test, y_train, y_test, baseline_params)
fpr_bayes, tpr_bayes, auc_bayes = evaluate_model(model_name, X_train, X_test, y_train, y_test, bayesian_params)
fpr_tpe, tpr_tpe, auc_tpe = evaluate_model(model_name, X_train, X_test, y_train, y_test, tpe_params)

# Comparison plot
plot_comparison([fpr_base, fpr_bayes, fpr_tpe], [tpr_base, tpr_bayes, tpr_tpe], [auc_base, auc_bayes, auc_tpe], ['Random Search', 'Bayesian Optimization', 'TPE'])


ModuleNotFoundError: No module named 'sklearn.optimization'